In [ ]:
pip install catboost

In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
train_df = pd.read_csv('/kaggle/input/banki-dataset/train_banki.csv')
train_df

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date,successful_utilization,treatment
0,NaN,1.0,Representatives,123.0,Первичная ДК,1,14.0,147000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-07-04,0,1
1,NaN,1.0,Offline,43.0,Airports,0,1.0,120000.0,PLT,0.0,...,1.0,0,1.0,0.0,-1.2,1.0,0,2024-06-06,0,1
2,0.0,1.0,Web,2.0,seo,0,NaN,15000.0,PLT,0.0,...,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-21,1,1
3,0.0,1.0,MB,2.0,One Click Offer,0,91.0,260000.0,PLT,0.0,...,1.0,0,NaN,1.0,-1.2,1.0,0,2024-05-23,0,1
4,0.0,1.0,Representatives,123.0,Первичная ДК,1,1.0,130000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-28,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,Representatives,123.0,Первичная ДК,1,34.0,95000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-13,1,1
297246,NaN,1.0,Representatives,123.0,Первичная ДК,1,1.0,83000.0,PLT,0.0,...,NaN,0,1.0,0.0,-1.2,NaN,0,2024-07-08,0,1
297247,0.0,1.0,Mobile Warm,2.0,TOOS cor clients,0,2.0,170000.0,PLT,0.0,...,1.0,0,NaN,0.0,1.0,1.0,0,2024-06-12,0,1
297248,0.0,1.0,Representatives,123.0,Первичная ДК,1,5.0,15000.0,PLT,0.0,...,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-02,1,1


In [3]:
test_df = pd.read_csv('/kaggle/input/banki-dataset/test_banki.csv')
test_df

,application_1,cc_1,cc_2,cc_3,cc_4,feature_1,mb_1,cc_5,cc_6,feature_2,...,bki_46,feature_24,cc_21,application_15,feature_25,feature_26,cc_22,partner_24,application_16,retro_date
0,0.0,1.0,NaN,NaN,NaN,0,6.0,NaN,NaN,0.0,...,373942.00,0.0,1.0,0,0.0,0.0,-1.2,1.0,0,2024-06-18
1,0.0,1.0,NaN,NaN,NaN,0,45.0,NaN,NaN,0.0,...,0.00,0.0,0.0,0,0.0,1.0,-1.2,1.0,0,2024-05-09
2,0.0,1.0,MB,2.0,Heavy xsell,0,13.0,299000.0,PLT,0.0,...,9724333.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-05-30
3,0.0,1.0,Representatives,123.0,SIM,1,158.0,140000.0,PLT,0.0,...,0.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-27
4,0.0,1.0,Representatives,123.0,Первичная ДК,0,36.0,170000.0,PLT,0.0,...,2237284.00,0.0,0.0,0,1.0,0.0,-1.2,1.0,0,2024-06-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74308,NaN,1.0,Representatives,123.0,Multitask,1,NaN,15000.0,PLT,0.0,...,2834099.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-05-15
74309,0.0,1.0,Representatives,123.0,Первичная ДК,1,12.0,95000.0,PLT,0.0,...,145886.00,NaN,NaN,0,NaN,0.0,-1.2,1.0,0,2024-07-10
74310,0.0,NaN,PreApprove (Mobile Applicants),2.0,Other,0,5.0,15000.0,PLT,0.0,...,5896358.00,0.0,0.0,1,0.0,1.0,-1.2,1.0,1,2024-06-27
74311,0.0,1.0,Representatives,123.0,Перевыпуск ДК,1,52.0,70000.0,PLT,0.0,...,5339.00,0.0,NaN,0,1.0,0.0,-1.2,1.0,0,2024-06-10


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297250 entries, 0 to 297249
Columns: 152 entries, application_1 to treatment
dtypes: float64(131), int64(17), object(4)
memory usage: 344.7+ MB


In [68]:
categorical_columns = train_df.select_dtypes(include=['int64']).columns
categorical_columns

Index(['feature_1', 'cc_10', 'cc_11', 'application_5', 'cc_12', 'cc_15',
       'application_6', 'application_7', 'application_9', 'cc_18',
       'application_11', 'application_13', 'application_14', 'application_15',
       'application_16', 'successful_utilization', 'treatment'],
      dtype='object')

In [14]:
# object_columns = train_df.select_dtypes(include=['object']).columns
# ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')

# # Применяем one-hot encoding к обучающей выборке
# encoded_train = pd.DataFrame(ohe.fit_transform(train_df[object_columns]), columns=ohe.get_feature_names_out(object_columns))
# encoded_test = pd.DataFrame(ohe.transform(test_df[object_columns]), columns=ohe.get_feature_names_out(object_columns))

# # Убираем оригинальные object-столбцы из исходных датасетов и добавляем закодированные данные
# train_df = train_df.drop(columns=object_columns).reset_index(drop=True)
# test_df = test_df.drop(columns=object_columns).reset_index(drop=True)

# train_df = pd.concat([train_df, encoded_train], axis=1)
# test_df = pd.concat([test_df, encoded_test], axis=1)

# Разделяем признаки и целевую переменную
X = train_df.drop(columns=['successful_utilization'])
y = train_df['successful_utilization']

# Разделяем данные на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

In [36]:
train_df

,application_1,cc_1,cc_3,feature_1,mb_1,cc_5,feature_2,cc_7,cc_8,application_2,...,retro_date_2024-07-18,retro_date_2024-07-19,retro_date_2024-07-20,retro_date_2024-07-21,retro_date_2024-07-22,retro_date_2024-07-23,retro_date_2024-07-24,retro_date_2024-07-25,retro_date_2024-07-26,retro_date_2024-07-27
0,NaN,1.0,123.0,1,14.0,147000.0,0.0,0.008663,0.008663,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1.0,43.0,0,1.0,120000.0,0.0,0.022936,0.022936,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,2.0,0,NaN,15000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,2.0,0,91.0,260000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,123.0,1,1.0,130000.0,0.0,0.029764,0.029764,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,123.0,1,34.0,95000.0,0.0,0.086722,0.086722,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297246,NaN,1.0,123.0,1,1.0,83000.0,0.0,0.025878,0.025878,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297247,0.0,1.0,2.0,0,2.0,170000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297248,0.0,1.0,123.0,1,5.0,15000.0,0.0,0.034553,0.034553,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train

,application_1,cc_1,cc_3,feature_1,mb_1,cc_5,feature_2,cc_7,cc_8,application_2,...,retro_date_2024-07-18,retro_date_2024-07-19,retro_date_2024-07-20,retro_date_2024-07-21,retro_date_2024-07-22,retro_date_2024-07-23,retro_date_2024-07-24,retro_date_2024-07-25,retro_date_2024-07-26,retro_date_2024-07-27
0,NaN,1.0,123.0,1,14.0,147000.0,0.0,0.008663,0.008663,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,NaN,1.0,43.0,0,1.0,120000.0,0.0,0.022936,0.022936,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,2.0,0,NaN,15000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,2.0,0,91.0,260000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,123.0,1,1.0,130000.0,0.0,0.029764,0.029764,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297245,NaN,1.0,123.0,1,34.0,95000.0,0.0,0.086722,0.086722,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297246,NaN,1.0,123.0,1,1.0,83000.0,0.0,0.025878,0.025878,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297247,0.0,1.0,2.0,0,2.0,170000.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297248,0.0,1.0,123.0,1,5.0,15000.0,0.0,0.034553,0.034553,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Заполняем пропущенные значения средними
# X_train.fillna(X_train.mean(), inplace=True)
# X_val.fillna(X_val.mean(), inplace=True)
# test_df.fillna(test_df.mean(), inplace=True)

X_train.fillna(0, inplace=True)
X_val.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)

In [ ]:
X_train

In [64]:
pip install hyperopt

Note: you may need to restart the kernel to use updated packages.


In [16]:
from catboost import CatBoostClassifier, Pool, metrics, cv
import numpy as np
import hyperopt

categorical = ["cc_2", "cc_4", "cc_6", "retro_date", "feature_1", 'feature_1', 
               'cc_10', 'cc_11', 'application_5', 'cc_12', 'cc_15',
       'application_6', 'application_7', 'application_9', 'cc_18',
       'application_11', 'application_13', 'application_14', 'application_15',
       'application_16', 'treatment']

def hyperopt_objective(params):
    model = CatBoostClassifier(
        auto_class_weights='SqrtBalanced',
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        depth=int(params['depth']),
        min_data_in_leaf=int(params['min_data_in_leaf']),
        eval_metric='AUC',
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
        iterations=500
    )

    cv_data = cv(
        Pool(X_val, y_val, cat_features=categorical), # вот тут тебе скорее всего надо cat_features изменить
        model.get_params(),
        logging_level='Verbose', # если хочешь видитеть прогресс
    )

    best_recall = np.max(cv_data['test-AUC-mean'])

    return 1 - best_recall


from numpy.random import Generator

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 3e-1),
    'depth': hyperopt.hp.choice('depth', [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0]), # в целом тут все хорошо сделано можешь не особо думать об этом
    'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf', 1, 100, 1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=60,  # тут можешь менять если можешь подольше позволить ему работать, настрой его примерно на длительность сна, с 60 он примерно 2 часа работает
    trials=trials,
    rstate=np.random.default_rng(42)
)

print(best)

Training on fold [0/3]                                

bestTest = 0.7173693772                               

bestIteration = 126                                   

Training on fold [1/3]                                

bestTest = 0.7237791744                               

bestIteration = 87                                    

Training on fold [2/3]                                

bestTest = 0.7180525048                               

bestIteration = 102                                   

Training on fold [0/3]                                                            

  2%|▏         | 1/60 [1:31:07<89:36:49, 5467.96s/trial, best loss: 0.280677794632307]


KeyboardInterrupt: 

In [58]:
# Обучаем модель CatBoost
catboost_model = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=0.133,
    verbose=False,
    eval_metric = 'AUC',
    use_best_model=True 
)

catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val), logging_level='Verbose')


bestTest = 0.7344455541
bestIteration = 380

Shrink model to first 381 iterations.


In [59]:
X_test = test_df.reindex(columns=X_train.columns, fill_value=0)

# Подготовка данных для предсказания с `treatment = 0` и `treatment = 1`
X_test_treatment_0 = X_test.copy()
X_test_treatment_0['treatment'] = 0

X_test_treatment_1 = X_test.copy()
X_test_treatment_1['treatment'] = 1

# Предсказываем вероятности
probs_treatment_0 = catboost_model.predict_proba(X_test_treatment_0)[:, 1]
probs_treatment_1 = catboost_model.predict_proba(X_test_treatment_1)[:, 1]

In [ ]:
probs_treatment_0

In [ ]:
probs_treatment_1

In [60]:
# Вычисляем uplift
uplift = probs_treatment_1 - probs_treatment_0

# Создаем DataFrame с нужной структурой
uplift_df = pd.DataFrame({
    'Unnamed: 0': range(len(uplift)),  # Индексы от 0 до количества строк
    'successful_utilization': uplift   # Результаты uplift
})

# Сохраняем в CSV
uplift_df.to_csv('uplift_predictions4.csv', index=False)

In [18]:
samp = pd.read_csv('/kaggle/input/banki-dataset/sample_submission_banki.csv')
samp

,Unnamed: 0,successful_utilization
0,0,-0.635423
1,1,-0.884969
2,2,0.000742
3,3,0.030837
4,4,-0.355528
...,...,...
74308,74308,0.640732
74309,74309,0.933332
74310,74310,-0.085592
74311,74311,-0.886694


In [35]:
my_pred = pd.read_csv('/kaggle/working/uplift_predictions2.csv')
my_pred

,Unnamed: 0,successful_utilization
0,0,1.199864e-02
1,1,3.744737e-07
2,2,7.790551e-02
3,3,4.250081e-02
4,4,6.442444e-02
...,...,...
74308,74308,3.613822e-02
74309,74309,1.040684e-01
74310,74310,5.882821e-02
74311,74311,3.598519e-02
